In [1]:
import os

In [2]:
%pwd

'd:\\Project\\P\\Mlops-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Project\\P\\Mlops-Project'

In [5]:
from MLOps_project.constant import *
from MLOps_project.utils.common import read_yaml, create_directories

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig: # class storage config of data ingestion
    root_dir: Path
    source_URL_label: str
    local_data_file: Path
    source_URL_for_image: Path
    source_code_download: Path

In [7]:
class ConfigurationManager: # manage: read and use files config
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        # create a root directory to storage data


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL_label=config.source_URL_label,
            local_data_file=config.local_data_file,
            source_URL_for_image = config.source_URL_for_image,
            source_code_download = config.source_code_download
        )
        # Helps get detailed configuration for data download.
        return data_ingestion_config

In [8]:
import os
import time
import urllib.request as request # use to download file from url
import subprocess
from MLOps_project import logger
from MLOps_project.utils.common import get_size

In [13]:
class DataIngestion: 
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(os.path.join(self.config.root_dir, "label_root.csv" )):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL_label,
                filename =os.path.join(self.config.root_dir, "label_root.csv" ))
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        if not os.path.exists(os.path.join(self.config.root_dir, "validation.lst" )):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL_for_image,
                filename =os.path.join(self.config.root_dir, "validation.lst" ))
            logger.info(f"{filename} download! with following info: \n{headers}")

        if not os.path.exists(os.path.join(self.config.root_dir, "download.py" )):
            filename, headers = request.urlretrieve(
                url = self.config.source_code_download,
                filename =os.path.join(self.config.root_dir, "download.py" ))
            logger.info(f"{filename} download! with following info: \n{headers}")

        #num_processes = 5 # số lượng tiến trình
        download_folder = self.config.local_data_file

        if not os.path.exists(download_folder):
            os.makedirs(download_folder)

        # Đường dẫn hoặc tên file danh sách hình ảnh
        IMAGE_LIST_FILE = "artifacts\data_ingestion\\validation.lst"

        command = [
            "python", "artifacts\data_ingestion\download.py",
            IMAGE_LIST_FILE,
            "--download_folder=" + download_folder,
            "--num_processes=5"
        ]

        # Chạy command
        print(subprocess.list2cmdline(command))
        try:
            subprocess.run(command, check=True)
        except subprocess.CalledProcessError as e:
            print("Error running the command:", e)


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2023-12-18 17:10:37,882: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-18 17:10:37,884: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-18 17:10:37,886: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-18 17:10:37,887: INFO: common: created directory at: artifacts]
[2023-12-18 17:10:37,889: INFO: common: created directory at: artifacts/data_ingestion]
python artifacts\data_ingestion\download.py artifacts\data_ingestion\validation.lst --download_folder=artifacts/data_ingestion/image --num_processes=5
